In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import dash_table
import numpy as np
from jupyter_dash import JupyterDash
#import pdfkit
import base64
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pyodbc
from urllib.parse import quote_plus
import io

In [2]:
# Parámetros de conexión
server = '10.0.0.133'
database = 'DB_GENERAL'
database2= 'DB_TRAMITE_DOCUMENTARIO'  # Reemplaza con el nombre de tu base de datos
username = 'sa'
password = 'Essalud23**'

def create_connection():
    conn = None
    try:
        # Establecer conexión
        conn = pyodbc.connect(r'DRIVER={ODBC Driver 18 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password};'
        'TrustServerCertificate=yes;')
        print("Conexión exitosa.")
    except Exception as e:
        print(f"Error al conectar: {e}")
    return conn

def create_connection2():
    conn = None
    try:
        # Establecer conexión
        conn = pyodbc.connect(r'DRIVER={ODBC Driver 18 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database2};'
        f'UID={username};'
        f'PWD={password};'
        'TrustServerCertificate=yes;')
        print("Conexión exitosa.")
    except Exception as e:
        print(f"Error al conectar: {e}")
    return conn

In [3]:
def fetch_data():
    conn = create_connection()
    if conn is None:
        raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
    try:
        query = f"""
SELECT 
    d.hoja_tramite,
    d.tipo_hoja,
    cdi.DESCRIPCION,
    d.INDICATIVO_OFICIO,
    p.RAZON_SOCIAL,
    d.FECHA_RECEPCION,
    d.ASUNTO,
    md.AUDIT_REC,
    do.SIGLAS AS SIGLAS_ORIGEN,
    do.DEPENDENCIA AS DEPENDENCIA_ORIGEN,
    dd.SIGLAS AS SIGLAS_DESTINO,
    dd.DEPENDENCIA AS DEPENDENCIA_DESTINO,
    ht.APELLIDOS_TRABAJADOR,
    ht.NOMBRES_TRABAJADOR,
    ed.ID_ESTADO_DOCUMENTO,
    ed.DESCRIPCION AS ESTADO_DOCUMENTO,
    md.AUDIT_TRAB_DER AS FECHA_DELEGADO,
    md.derivado,
    md.finalizado
FROM 
    DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO md
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO d ON md.ID_DOCUMENTO = d.ID_DOCUMENTO
LEFT OUTER JOIN 
    DB_GENERAL.dbo.PERSONA p ON d.ID_PERSONA = p.ID
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.dbo.CLASE_DOCUMENTO_INTERNO cdi ON d.ID_CLASE_DOCUMENTO_INTERNO = cdi.ID_CLASE_DOCUMENTO_INTERNO
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.dbo.ESTADO_DOCUMENTO ed ON ed.ID_ESTADO_DOCUMENTO = d.ID_ESTADO_DOCUMENTO
LEFT OUTER JOIN 
    DB_GENERAL.jcardenas.H_DEPENDENCIA do ON do.CODIGO_DEPENDENCIA = md.ID_DEPENDENCIA_ORIGEN 
LEFT OUTER JOIN 
    DB_GENERAL.jcardenas.H_DEPENDENCIA dd ON dd.CODIGO_DEPENDENCIA = md.ID_DEPENDENCIA_DESTINO
LEFT JOIN 
    DB_GENERAL.jcardenas.H_TRABAJADOR ht ON ht.CODIGO_TRABAJADOR = md.codigo_trabajador
WHERE
    (do.SIGLAS LIKE 'SG' OR dd.SIGLAS LIKE 'SG')
    AND
    md.AUDIT_TRAB_DER = (
        SELECT MAX(sub_md.AUDIT_TRAB_DER)
        FROM DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO sub_md
        LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO sub_d ON sub_md.ID_DOCUMENTO = sub_d.ID_DOCUMENTO
        WHERE sub_d.hoja_tramite = d.hoja_tramite
    )
        """
        result = pd.read_sql(query, conn)
        result['estado']= np.where((result['ID_ESTADO_DOCUMENTO'] != 2) & (result['ID_ESTADO_DOCUMENTO'] != 10) & (result['derivado'] == 0) & (result['finalizado'] == 0),'Pendiente','Otros')
        result = result[result['estado']=='Pendiente']


    finally:
        conn.close()  # Cerrar la conexión al final
    return result

In [13]:
external_stylesheets = [dbc.themes.BOOTSTRAP,
    'https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css',  # Bootstrap CSS
    'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.1/css/all.min.css']  # Bootstrap Icons
# Crear la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
    # Encabezado
    dbc.Row([
        dbc.Col([
            html.H1("var: Seguimiento del trámite", style={
                'color': '#0064AF',
                'fontSize': '28px',
                'textAlign': 'left',
                'fontWeight': 'bold',
                'fontFamily': 'Calibri'
            }),
            html.H2("Fuente: SGD", style={
                'color': '#0064AF',
                'fontSize': '12px',
                'textAlign': 'left',
                'fontFamily': 'Calibri'
            })
        ], width=12),
    ], style={'margin-top': '20px'}),

    # Input y botón de búsqueda
    dbc.Row([
        dbc.Col([
            html.H6("# hoja de trámite - # Hoja de tramite", style={
                'fontSize': '14px',
                'color': '#606060',
                'fontWeight': 'normal',
                'fontFamily': 'Calibri'
            }),
            dcc.Input(
                id='hoja_tramite',
                type='text',
                placeholder='Número de Expediente',
                style={'width': '80%', 'fontSize': '14px', 'display': 'inline-block'},
                debounce=True  # Esto permite activar el callback al presionar Enter
            ),
            dbc.Button(
                html.I(className="fas fa-search"),
                id='search-button',
                style={'background-color': '#0064AF', 'border-color': '#0064AF', 'color': 'white'},
                className='align-middle'
            )
        ], width=12, md=12, lg=5, className='mb-2'),
    ], style={'margin-top': '20px'}),

    # Contenedor para el spinner y la tabla
    dbc.Row([
        dbc.Col([
            dbc.Spinner(
                id='spinner',
                color="primary",
                size="lg",
                children=html.Div(id='table_container'),
                fullscreen=True  # Make the spinner fullscreen
            )
        ], width=12)
    ], style={'margin-top': '20px'}),

], fluid=True)

@app.callback(
    [Output('spinner', 'children'),
     Output('spinner', 'fullscreen')],
    [Input('search-button', 'n_clicks')],
    [dash.dependencies.State('hoja_tramite', 'value')]
)
def update_table(n_clicks, hoja_tramite_value):
    # Cargar los datos
    df = fetch_data()

    # Verificar si se ha clicado el botón de búsqueda
    if n_clicks is None or n_clicks == 0:
        df_filtered = df  # Mostrar todos los datos por defecto
    elif hoja_tramite_value and hoja_tramite_value.strip():
        # Filtrar datos si el valor de búsqueda no está vacío
        df_filtered = df[df['hoja_tramite'] == hoja_tramite_value]
    else:
        df_filtered = df  # Mostrar todos los datos si el valor de búsqueda está vacío

    if not df_filtered.empty:
        # Crear la DataTable
        table = dash_table.DataTable(
            id='data_table',
            columns=[{"name": i, "id": i} for i in df_filtered.columns],
            data=df_filtered.to_dict('records'),
            style_table={'overflowX': 'auto'},
            style_cell={
                'textAlign': 'left',
                'fontFamily': 'Calibri',
                'padding': '5px'
            },
            style_header={
                'backgroundColor': '#0064AF',
                'fontWeight': 'bold',
                'color': 'white'
            },
            page_size=20  # Número de filas por página
        )
        return table, False
    else:
        # Devuelve un mensaje indicando que no hay datos y que el spinner no esté en pantalla completa
        return html.Div("No data available", style={'color': 'red', 'fontWeight': 'bold'}), False

if __name__ == '__main__':
    app.run_server(debug=True)

Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_10280\4048390100.py:53: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_10280\4048390100.py:53: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Conexión exitosa.


...

C:\Users\kings\AppData\Local\Temp\ipykernel_10280\4048390100.py:53: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_10280\4048390100.py:53: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_10280\4048390100.py:53: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

